In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.optimize import minimize

In [65]:
n = 20000
period = 2

gamma = 6
rho = 0.2

market_mu = 0.12
market_sigma = 0.3

rf_mu = 0.04
rf_sigma = 0.01

initial_wealth = 150000
initial_income = 13000

In [66]:
def max_utility(n, period, rho, gamma, market_mu, market_sigma, rf_mu, rf_sigma):
    def obj(weight, n, period, rho, gamma, market_mu, market_sigma, rf_mu, rf_sigma):
        np.random.seed(1)

        market_ = stats.norm.rvs(size=(n, period+1), loc=0, scale=1)
        rf_ = stats.norm.rvs(size=(n, period+1), loc=0, scale=1)
        brownian_ = stats.norm.rvs(size=(n, period+1), loc=0, scale=1)
        hc_ = rho * market_ + np.sqrt(1 - rho ** 2) * brownian_

        risky_ret = np.exp(market_mu+(0.5*(market_sigma**2))+market_sigma*market_[:, 0])
        rf_ret = np.exp(rf_mu+(0.5*(rf_sigma**2))+rf_sigma*rf_[:, 0])
        #rf_ret = np.exp(rf_mu*np.ones(market_[:, 0].shape))

        h_ret = initial_income*np.cumprod(np.exp(0.04+0.03*(rho * market_[:, :-1] + np.sqrt(1 - rho ** 2) * hc_[:, :-1])), axis=1)
        dc_rate = np.exp((0.04 + 0.04) * (np.array(range(period)) + 1) * (-1))

        W_t = (initial_wealth+(initial_income*0.3))*(np.array([risky_ret, rf_ret]).T @ weight)
        
        if period == 0:
            H_t = np.zeros([W_t.shape[0]])
        else:
            H_t = h_ret @ dc_rate

        if gamma == 1:
            obj = np.log(W_t+H_t) * (-1)
        else:
            obj = (((W_t+H_t) ** (1-gamma)) / (1-gamma)) * (-1)

        return np.mean(obj)

    x = np.ones([2]) / 2
    cons = ({'type': 'ineq', 'fun': lambda x: 1 - np.sum(x)},
            {'type': 'ineq', 'fun': lambda x: x},)
    optimized = minimize(obj, x, (n, period, rho, gamma, market_mu, market_sigma, rf_mu, rf_sigma), method='COBYLA', 
                         constraints=cons)
    if not optimized.success: raise BaseException(optimized.message)
    return optimized.x

In [67]:
max_utility(n, period, rho, gamma, market_mu, market_sigma, rf_mu, rf_sigma)

array([0.35260221, 0.64739779])

In [244]:
class Maximize(object):
    np.random.seed(1)
    period = 30
    
    def __init__(self, n, rho):
        self.n = n
        #self.period = period
        self.rho = rho
        
        
    def MC_data(self):
        self.market_ = stats.norm.rvs(size=(self.n, self.period+1), loc=0, scale=1)
        self.rf_ = stats.norm.rvs(size=(self.n, self.period+1), loc=0, scale=1)
        self.brownian_ = stats.norm.rvs(size=(self.n, self.period+1), loc=0, scale=1)
        self.hc_ = self.rho * self.market_ + np.sqrt(1 - self.rho ** 2) * self.brownian_

        
    def get_human_capital(self):
        h_ret = initial_income*np.cumprod(np.exp(0.04+0.03*(self.rho * self.market_[:, :-1] + np.sqrt(1 - self.rho ** 2) * self.hc_[:, :-1])), axis=1)
        dc_rate = np.exp((0.04 + 0.04) * (np.array(range(self.period)) + 1) * (-1))
        return h_ret @ dc_rate
    
    
    def get_asset_ret(self, market_mu, market_sigma, rf_mu, rf_sigma):
        risky_ret = np.exp(market_mu+(0.5*(market_sigma**2))+market_sigma*self.market_[:, 0])
        rf_ret = np.exp(rf_mu+(0.5*(rf_sigma**2))+rf_sigma*self.rf_[:, 0])
        #rf_ret = np.exp(rf_mu*np.ones(market_[:, 0].shape))
        return np.array([risky_ret, rf_ret]).T
    
    
    def max_utility(self, gamma, market_mu, market_sigma, rf_mu, rf_sigma):
        def obj(weight, n, period, rho, gamma, market_mu, market_sigma, rf_mu, rf_sigma):
            risky_ret = np.exp(market_mu+(0.5*(market_sigma**2))+market_sigma*market_[:, 0])
            rf_ret = np.exp(rf_mu+(0.5*(rf_sigma**2))+rf_sigma*rf_[:, 0])
            #rf_ret = np.exp(rf_mu*np.ones(market_[:, 0].shape))

            h_ret = initial_income*np.cumprod(np.exp(0.04+0.03*(rho * market_[:, :-1] + np.sqrt(1 - rho ** 2) * hc_[:, :-1])), axis=1)
            dc_rate = np.exp((0.04 + 0.04) * (np.array(range(period)) + 1) * (-1))

            W_t = (initial_wealth+(initial_income*0.3))*(np.array([risky_ret, rf_ret]).T @ weight)

            if period == 0:
                H_t = np.zeros([W_t.shape[0]])
            else:
                H_t = h_ret @ dc_rate

            if gamma == 1:
                obj = np.log(W_t+H_t) * (-1)
            else:
                obj = (((W_t+H_t) ** (1-gamma)) / (1-gamma)) * (-1)

            return np.mean(obj)

        x = np.ones([2]) / 2
        cons = ({'type': 'ineq', 'fun': lambda x: 1 - np.sum(x)},
                {'type': 'ineq', 'fun': lambda x: x},)
        optimized = minimize(obj, x, (n, period, rho, gamma, market_mu, market_sigma, rf_mu, rf_sigma), method='COBYLA', 
                             constraints=cons)
        if not optimized.success: raise BaseException(optimized.message)
        return optimized.x
    
        
    @classmethod
    def change_period(cls, period):
        cls.period = period

In [341]:
np.zeros([np.array(range(0)).shape[0]])

array([], dtype=float64)

In [182]:
class Maximize(object):
    period = 30
    np.random.seed(period)

    
    def __init__(self, n, rho, gamma):
        self.n = n
        #self.period = period
        self.rho = rho
        self.gamma = gamma
        #self.initial_wealth = initial_wealth
        #self.initial_income = initial_income
        
        
    def MC_data(self):
        self.market_ = stats.norm.rvs(size=(self.n, self.period+1), loc=0, scale=1)
        self.rf_ = stats.norm.rvs(size=(self.n, self.period+1), loc=0, scale=1)
        self.brownian_ = stats.norm.rvs(size=(self.n, self.period+1), loc=0, scale=1)
        self.hc_ = self.rho * self.market_ + np.sqrt(1 - self.rho ** 2) * self.brownian_

        
    def get_original_H(self, hc_mu, hc_sigma):
        dc_rate = np.exp((0.04 + 0.04 + 0.04) * (np.array(range(self.period)) + 1) * (-1))
        
        if self.period == 0:
            self.labor_income = np.zeros([self.n, self.period+1])
            self.H_t = np.zeros([self.n])
        else:
            self.labor_income = self.initial_income*np.cumprod(np.exp(hc_mu+hc_sigma*(self.rho * self.market_[:, :-1] + np.sqrt(1 - self.rho ** 2) * self.hc_[:, :-1])), axis=1)
            self.H_t = self.labor_income @ dc_rate
    
    
    def get_asset_ret(self, market_mu, market_sigma, rf_mu, rf_sigma):
        risky_ret = np.exp(market_mu+(0.5*(market_sigma**2))+market_sigma*self.market_[:, 0])
        rf_ret = np.exp(rf_mu+(0.5*(rf_sigma**2))+rf_sigma*self.rf_[:, 0])
        #rf_ret = np.exp(rf_mu*np.ones(market_[:, 0].shape))
        
        self.R_t =  np.array([risky_ret, rf_ret]).T
    
    
    def fit(self):
        def obj(weight, gamma):
            W_t = (self.initial_wealth+(self.initial_income*0.3))*(self.R_t @ weight)

            if self.gamma == 1:
                obj = np.log(W_t+self.H_t) * (-1)
            else:
                obj = (((W_t+self.H_t) ** (1-self.gamma)) / (1-self.gamma)) * (-1)

            return np.mean(obj)

        x = np.ones([self.R_t.shape[1]]) / self.R_t.shape[1]
        cons = ({'type': 'ineq', 'fun': lambda x: 1 - np.sum(x)},
                {'type': 'ineq', 'fun': lambda x: x},)
        optimized = minimize(obj, x, (self.gamma), method='COBYLA', constraints=cons)
        if not optimized.success: raise BaseException(optimized.message)
        return np.round(optimized.x, 4)
    
        
    def change_parameters(self, period, initial_wealth, initial_income):
        self.period = period
        self.initial_wealth = initial_wealth
        self.initial_income = initial_income

In [187]:
model = Maximize(n=20000, rho=0.3, gamma=6)
for i in range(31)[::-1]:
    
    if i is range(31)[::-1][0]:
        model.change_parameters(i, 1000, 3000)
    else:
        model.change_parameters(i, model.initial_wealth * np.mean(model.R_t @ model.fit()), np.mean(model.labor_income[:, 0]))

    model.MC_data()
    model.get_original_H(hc_mu=0.05, hc_sigma=0.05)
    model.get_asset_ret(market_mu=0.12, market_sigma=0.35, rf_mu=0.04, rf_sigma=0.03)
    print(model.period, model.fit() * 100, np.round(model.initial_wealth), np.round(np.mean(model.labor_income[:, 0])))


30 [100.   0.] 1000 3159.0
29 [100.   0.] 1277.0 3324.0
28 [100.   0.] 1627.0 3502.0
27 [100.   0.] 2075.0 3685.0
26 [100.   0.] 2629.0 3880.0
25 [100.   0.] 3350.0 4087.0
24 [100.   0.] 4280.0 4304.0
23 [100.   0.] 5453.0 4532.0
22 [100.   0.] 6959.0 4767.0
21 [100.   0.] 8863.0 5019.0
20 [95.14  4.86] 11254.0 5288.0
19 [84.83 15.17] 14232.0 5569.0
18 [74.6 25.4] 17662.0 5860.0
17 [69.07 30.93] 21406.0 6172.0
16 [63.58 36.42] 25792.0 6501.0
15 [56.22 43.78] 30677.0 6844.0
14 [52.77 47.23] 35931.0 7203.0
13 [51.75 48.25] 41852.0 7582.0
12 [48.03 51.97] 48730.0 7982.0
11 [45.97 54.03] 56225.0 8402.0
10 [42.9 57.1] 64644.0 8844.0
9 [41.06 58.94] 73720.0 9314.0
8 [38.79 61.21] 83953.0 9810.0
7 [37.45 62.55] 95013.0 10326.0
6 [34.78 65.22] 107309.0 10872.0
5 [33.5 66.5] 120267.0 11444.0
4 [32.08 67.92] 134582.0 12055.0
3 [30.7 69.3] 150406.0 12681.0
2 [29.63 70.37] 167259.0 13357.0
1 [28.25 71.75] 185548.0 14062.0
0 [27.01 72.99] 205327.0 0.0


In [ ]:
class Maximize(object):
    period = 30
    np.random.seed(30)
    
    def __init__(self, n, rho, gamma, initial_wealth, initial_income):
        self.n = n
        self.rho = rho
        self.gamma = gamma
        self.initial_wealth = initial_wealth
        self.initial_income = initial_income
        
        
    def MC_data(self):
        self.market_ = stats.norm.rvs(size=(self.n, self.period+1), loc=0, scale=1)
        self.rf_ = stats.norm.rvs(size=(self.n, self.period+1), loc=0, scale=1)
        self.brownian_ = stats.norm.rvs(size=(self.n, self.period+1), loc=0, scale=1)
        self.hc_ = self.rho * self.market_ + np.sqrt(1 - self.rho ** 2) * self.brownian_

        
    def get_human_capital(self):
        self.labor_income = self.initial_income*np.cumprod(np.exp(0.04+0.03*(self.rho * self.market_[:, :-1] + np.sqrt(1 - self.rho ** 2) * self.hc_[:, :-1])), axis=1)
        dc_rate = np.exp((0.04 + 0.04) * (np.array(range(self.period)) + 1) * (-1))
        
        if self.period == 0:
            self.H_t = np.zeros([self.n])
        else:
            self.H_t = self.labor_income @ dc_rate
    
    
    def get_asset_ret(self, market_mu, market_sigma, rf_mu, rf_sigma):
        risky_ret = np.exp(market_mu+(0.5*(market_sigma**2))+market_sigma*self.market_[:, 0])
        rf_ret = np.exp(rf_mu+(0.5*(rf_sigma**2))+rf_sigma*self.rf_[:, 0])
        #rf_ret = np.exp(rf_mu*np.ones(market_[:, 0].shape))
        
        self.R_t =  np.array([risky_ret, rf_ret]).T
    
    
    def fit(self):
        def obj(weight, gamma):
            W_t = (self.initial_wealth+(self.initial_income*0.3))*(self.R_t @ weight)

            if self.gamma == 1:
                obj = np.log(W_t+self.H_t) * (-1)
            else:
                obj = (((W_t+self.H_t) ** (1-self.gamma)) / (1-self.gamma)) * (-1)

            return np.mean(obj)

        x = np.ones([self.R_t.shape[1]]) / self.R_t.shape[1]
        cons = ({'type': 'ineq', 'fun': lambda x: 1 - np.sum(x)},
                {'type': 'ineq', 'fun': lambda x: x},)
        optimized = minimize(obj, x, (self.gamma), method='COBYLA', constraints=cons)
        if not optimized.success: raise BaseException(optimized.message)
        return np.round(optimized.x, 5)
    
        
    @classmethod
    def change_period(cls, period):
        cls.period = period
        
    def change_financial(cls, initial_wealth=None, initial_income=None):
        cls.initial_wealth = initial_wealth
        cls.initial_income = initial_income

In [245]:
model = Maximize(20000, 0.3)

In [246]:
model.period

30

In [250]:
model.MC_data()
model.market_.shape

(20000, 31)

In [251]:
model.change_period(25)

model.MC_data()
model.market_.shape

(20000, 26)

In [242]:
model.change_period(25)
model.period

25

In [243]:
model.sigma

<bound method Maximize.sigma of <__main__.Maximize object at 0x0000021A9F9BB748>>

In [140]:
model.get_asset_ret(0.1, 0.2, 0.05, 0)

array([[1.11124324, 1.0512711 ],
       [0.92759281, 1.0512711 ],
       [1.18633074, 1.0512711 ],
       ...,
       [1.21427925, 1.0512711 ],
       [1.1285435 , 1.0512711 ],
       [1.00479352, 1.0512711 ]])

In [ ]:
class Maximize(object):
    period = 30
    

    @classmethod
    def change_period(cls, period):
        cls.period = period
    

class Optimize(Maximize):    
    def __init__(self, n, rho):
        self.n = n
        self.rho = rho
        
        self.market_ = stats.norm.rvs(size=(self.n, self.period+1), loc=0, scale=1)
        self.rf_ = stats.norm.rvs(size=(self.n, self.period+1), loc=0, scale=1)
        self.brownian_ = stats.norm.rvs(size=(self.n, self.period+1), loc=0, scale=1)
        self.hc_ = self.rho * self.market_ + np.sqrt(1 - self.rho ** 2) * self.brownian_
    
    def get_human_capital(self):
        h_ret = initial_income*np.cumprod(np.exp(0.04+0.03*(self.rho * self.market_[:, :-1] + np.sqrt(1 - self.rho ** 2) * self.hc_[:, :-1])), axis=1)
        dc_rate = np.exp((0.04 + 0.04) * (np.array(range(self.period)) + 1) * (-1))
        return h_ret @ dc_rate
    
    
    def get_asset_ret(self, market_mu, market_sigma, rf_mu, rf_sigma):
        risky_ret = np.exp(market_mu+(0.5*(market_sigma**2))+market_sigma*self.market_[:, 0])
        rf_ret = np.exp(rf_mu+(0.5*(rf_sigma**2))+rf_sigma*self.rf_[:, 0])
        return np.array([risky_ret, rf_ret]).T
    
    
    